<a href="https://colab.research.google.com/github/LimPark996/metal-defect-synthesis/blob/main/V0/metal_defect_gradio_demo_LlamaGen_Halton(PoCFinal).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔧 Metal Defect Inpainting - Gradio Demo

금속 표면 결함 Inpainting 데모

**사전 조건:**
- VQGAN Fine-tuning 완료 (`metal_defect_synthesis.ipynb`)
- MaskGIT 학습 완료 (`metal_defect_MaskGit.ipynb`)
- Google Drive에 체크포인트 저장됨

## 1. 환경 설정

In [ ]:
!pip install gradio omegaconf einops -q

# LlamaGen 클론
!git clone https://github.com/FoundationVision/LlamaGen.git

# Halton-MaskGIT 클론 (선택: halton sequence 함수만 필요하면 직접 정의해도 됨)
!git clone https://github.com/valeoai/Halton-MaskGIT.git

Cloning into 'LlamaGen'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 191 (delta 59), reused 38 (delta 38), pack-reused 97 (from 1)
Receiving objects: 100% (191/191), 5.49 MiB | 5.79 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Cloning into 'Halton-MaskGIT'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 247 (delta 51), reused 38 (delta 38), pack-reused 173 (from 1)
Receiving objects: 100% (247/247), 14.11 MiB | 15.80 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# GPU 확인
!nvidia-smi

Fri Dec 12 08:42:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             56W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2. 경로 설정

In [ ]:
import sys
import os

sys.path.append('/content/LlamaGen')

# LlamaGen VQGAN (config 필요 없음!)
VQGAN_CKPT_PATH = "/content/drive/MyDrive/metal-defect/LlamaGen/checkpoints/vqgan_finetune_up_epoch50.pt"

# Halton MaskGIT
MASKGIT_CKPT_PATH = "/content/drive/MyDrive/metal-defect/halton-maskgit/checkpoints/maskgit_epoch100.pt"

## 3. 설정 및 상수 정의

In [ ]:
# =============================================================================
# 설정 및 상수
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image
import torchvision.transforms as T
import math

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Device: {device}")

# 이미지 설정
IMAGE_SIZE = 256
LATENT_SIZE = 16  # 256 / 16 = 16
CODEBOOK_SIZE = 16384
CODEBOOK_DIM = 8       # LlamaGen은 8차원! (taming은 256)
LATENT_SIZE = 16
SEQ_LEN = 256
NUM_CLASSES = 6
MASK_TOKEN_ID = 16384  # vocab_size와 동일

# 클래스 정보 (NEU-DET + GC10-DET)
CLASS_NAMES = ['inclusion', 'pit_hole', 'scratch_crease', 'spot_stain', 'line_crack', 'fold_scale']

CLASS_NAMES_KR = [
    '0: 개재물 (Inclusion)',
    '1: 피트홀 (Pit Hole)',
    '2: 스크래치/주름 (Scratch/Crease)',
    '3: 얼룩 (Spot/Stain)',
    '4: 균열 (Line/Crack)',
    '5: 접힘/스케일 (Fold/Scale)',
]

NUM_CLASSES = 6

# 마스크 프리셋
def get_mask_preset(preset_name):
    """마스크 프리셋 반환"""
    presets = {
        "중앙 (작음) 6x6": [],
        "중앙 (큼) 8x8": [],
        "좌상단 6x6": [],
        "우하단 6x6": []
    }

    # 중앙 (작음) 6x6
    for y in range(5, 11):
        for x in range(5, 11):
            presets["중앙 (작음) 6x6"].append(y * 16 + x)

    # 중앙 (큼) 8x8
    for y in range(4, 12):
        for x in range(4, 12):
            presets["중앙 (큼) 8x8"].append(y * 16 + x)

    # 좌상단 6x6
    for y in range(0, 6):
        for x in range(0, 6):
            presets["좌상단 6x6"].append(y * 16 + x)

    # 우하단 6x6
    for y in range(10, 16):
        for x in range(10, 16):
            presets["우하단 6x6"].append(y * 16 + x)

    return presets.get(preset_name, presets["중앙 (작음) 6x6"])

🖥️ Device: cuda


## 4. 모델 정의

In [ ]:
# ============================================
# Halton-MaskGIT Transformer 구현
# ============================================

from einops import rearrange


# ----- 기본 구성요소 -----

def modulate(x, shift, scale):
    """AdaLayerNorm의 modulation 연산"""
    return x * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)


class RMSNorm(nn.Module):
    """Root Mean Square Layer Normalization"""
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        norm = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        return self.weight * norm


class SwiGLU(nn.Module):
    """SwiGLU FFN (LLaMA style)"""
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        # hidden_dim 조정 (SwiGLU는 2/3배 사용)
        hidden_dim = int(2 * hidden_dim / 3)
        # 256의 배수로 맞춤
        hidden_dim = 256 * ((hidden_dim + 255) // 256)

        self.w1 = nn.Linear(dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.w3 = nn.Linear(dim, hidden_dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # SwiGLU: SiLU(W1(x)) * W3(x)
        x = F.silu(self.w1(x)) * self.w3(x)
        x = self.dropout(x)
        x = self.w2(x)
        return x


class QKNorm(nn.Module):
    """Query/Key Normalization for stable attention"""
    def __init__(self, dim):
        super().__init__()
        self.q_norm = RMSNorm(dim)
        self.k_norm = RMSNorm(dim)

    def forward(self, q, k):
        return self.q_norm(q), self.k_norm(k)


class Attention(nn.Module):
    """Multi-Head Self-Attention with QK Norm and Flash Attention"""
    def __init__(self, dim, num_heads, dropout=0.):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.wq = nn.Linear(dim, dim, bias=False)
        self.wk = nn.Linear(dim, dim, bias=False)
        self.wv = nn.Linear(dim, dim, bias=False)
        self.wo = nn.Linear(dim, dim, bias=False)

        self.qk_norm = QKNorm(dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, N, C = x.shape

        # Q, K, V projection
        q, k, v = self.wq(x), self.wk(x), self.wv(x)

        # QK Normalization
        q, k = self.qk_norm(q, k)

        # Reshape for multi-head
        q = q.view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, N, self.num_heads, self.head_dim).transpose(1, 2)

        # Flash Attention (PyTorch 2.0+)
        out = F.scaled_dot_product_attention(
            q, k, v,
            dropout_p=self.dropout.p if self.training else 0.
        )

        # Reshape back
        out = out.transpose(1, 2).contiguous().view(B, N, C)
        out = self.wo(out)

        return out


class TransformerBlock(nn.Module):
    """Transformer Block with AdaLayerNorm"""
    def __init__(self, dim, num_heads, mlp_ratio=4., dropout=0.):
        super().__init__()

        # AdaLN modulation MLP
        # 6개 파라미터: gamma1, beta1, alpha1, gamma2, beta2, alpha2
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(dim, dim * 6)
        )

        # Attention
        self.norm1 = RMSNorm(dim)
        self.attn = Attention(dim, num_heads, dropout)

        # FFN (SwiGLU)
        self.norm2 = RMSNorm(dim)
        self.ffn = SwiGLU(dim, int(dim * mlp_ratio), dropout)

    def forward(self, x, cond):
        """
        Args:
            x: [B, N, C] 토큰 임베딩
            cond: [B, C] 클래스 조건 임베딩
        """
        # AdaLN modulation 파라미터 계산
        gamma1, beta1, alpha1, gamma2, beta2, alpha2 = \
            self.adaLN_modulation(cond).chunk(6, dim=-1)

        # Attention with AdaLN
        x = x + alpha1.unsqueeze(1) * self.attn(
            modulate(self.norm1(x), beta1, gamma1)
        )

        # FFN with AdaLN
        x = x + alpha2.unsqueeze(1) * self.ffn(
            modulate(self.norm2(x), beta2, gamma2)
        )

        return x


class AdaNorm(nn.Module):
    """Final AdaLayerNorm before output"""
    def __init__(self, dim):
        super().__init__()
        self.norm = RMSNorm(dim)
        self.adaLN = nn.Sequential(
            nn.SiLU(),
            nn.Linear(dim, dim * 2)
        )

    def forward(self, x, cond):
        shift, scale = self.adaLN(cond).chunk(2, dim=-1)
        return modulate(self.norm(x), shift, scale)


print("✅ 기본 구성요소 정의 완료")

✅ 기본 구성요소 정의 완료


In [ ]:
# ============================================
# MaskGIT Transformer 메인 클래스
# ============================================

class MaskGITTransformer(nn.Module):
    """
    Halton-MaskGIT 스타일 Bidirectional Transformer

    특징:
    - AdaLayerNorm으로 클래스 조건 주입
    - SwiGLU FFN
    - QK Normalization
    - Weight Tying (tok_emb ↔ head)
    """

    def __init__(
        self,
        vocab_size=16384,      # codebook 크기
        seq_len=256,           # 토큰 시퀀스 길이 (16*16)
        hidden_dim=512,        # Transformer hidden 차원
        num_layers=12,         # Transformer 레이어 수
        num_heads=8,           # Multi-head attention의 head 수
        mlp_ratio=4.,          # FFN hidden 배율
        dropout=0.1,           # Dropout 비율
        num_classes=6,         # 클래스 수
    ):
        super().__init__()

        self.vocab_size = vocab_size
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.mask_token_id = vocab_size  # [MASK] = 16384

        # ===== Embeddings =====
        # 토큰 임베딩: 0~16383 (codebook) + 16384 ([MASK])
        self.tok_emb = nn.Embedding(vocab_size + 1, hidden_dim)

        # 위치 임베딩: 0~255 (16x16 위치)
        self.pos_emb = nn.Embedding(seq_len, hidden_dim)

        # 클래스 임베딩: 0~5 (결함 클래스) + 6 (CFG용 null class)
        self.cls_emb = nn.Embedding(num_classes + 1, hidden_dim)

        # ===== Transformer Blocks =====
        self.blocks = nn.ModuleList([
            TransformerBlock(hidden_dim, num_heads, mlp_ratio, dropout)
            for _ in range(num_layers)
        ])

        # ===== Output =====
        self.final_norm = AdaNorm(hidden_dim)

        # 출력 헤드 (Weight Tying)
        self.head = nn.Linear(hidden_dim, vocab_size + 1, bias=False)
        self.head.weight = self.tok_emb.weight  # ⭐ Weight Tying!

        # 가중치 초기화
        self._init_weights()

    def _init_weights(self):
        """가중치 초기화"""
        # 임베딩 초기화
        nn.init.normal_(self.tok_emb.weight, std=0.02)
        nn.init.normal_(self.pos_emb.weight, std=0.02)
        nn.init.normal_(self.cls_emb.weight, std=0.02)

        # AdaLN modulation 레이어 zero 초기화
        for block in self.blocks:
            nn.init.zeros_(block.adaLN_modulation[1].weight)
            nn.init.zeros_(block.adaLN_modulation[1].bias)

    def forward(self, x, y, drop_label=None):
        """
        순전파

        Args:
            x: [B, H, W] 또는 [B, N] - 마스킹된 토큰 (일부가 16384)
            y: [B] - 클래스 라벨 (0~5)
            drop_label: [B] bool - True면 해당 샘플의 클래스 조건 무시 (CFG용)

        Returns:
            logits: [B, N, vocab_size+1] - 각 위치의 예측 확률
        """
        B = x.shape[0]

        # [B, H, W] → [B, N] 변환
        if x.dim() == 3:
            x = x.view(B, -1)

        # ===== 클래스 조건 처리 =====
        if drop_label is not None:
            # drop_label이 True인 샘플은 null class (num_classes)로 대체
            y = torch.where(drop_label, torch.full_like(y, self.num_classes), y)

        # 클래스 임베딩: [B] → [B, hidden_dim]
        cond = self.cls_emb(y)

        # ===== 토큰 + 위치 임베딩 =====
        # 위치 인덱스 생성
        pos = torch.arange(self.seq_len, device=x.device)

        # 임베딩: [B, N] → [B, N, hidden_dim]
        x = self.tok_emb(x) + self.pos_emb(pos)

        # ===== Transformer 블록 통과 =====
        for block in self.blocks:
            x = block(x, cond)

        # ===== 출력 =====
        x = self.final_norm(x, cond)
        logits = self.head(x)  # [B, N, vocab_size+1]

        return logits


print("✅ MaskGITTransformer 클래스 정의 완료")

✅ MaskGITTransformer 클래스 정의 완료


## 5. 모델 로딩

In [ ]:
# =============================================================================
# VQGAN 로딩
# =============================================================================

from tokenizer.tokenizer_image.vq_model import VQ_models

# LlamaGen VQ-16 로드
vqgan = VQ_models["VQ-16"](
    codebook_size=CODEBOOK_SIZE,     # 16384
    codebook_embed_dim=CODEBOOK_DIM  # 8
)

# 체크포인트 로드
checkpoint = torch.load(VQGAN_CKPT_PATH, map_location=device)
if "vqmodel" in checkpoint:
    state_dict = checkpoint["vqmodel"]
else:
    state_dict = checkpoint.get("state_dict", checkpoint)

vqgan.load_state_dict(state_dict, strict=True)
vqgan = vqgan.to(device)
vqgan.eval()
for param in vqgan.parameters():
    param.requires_grad = False

In [ ]:
print("🔄 MaskGIT 로딩 중...")

# 체크포인트 로드
ckpt = torch.load(MASKGIT_CKPT_PATH, map_location=device, weights_only=False)

# config 확인
if 'config' in ckpt:
    model_config = ckpt['config'].copy()  # 원본 수정 방지

    # MaskGITTransformer가 받지 않는 키 제거
    keys_to_remove = ['model_size']
    for key in keys_to_remove:
        model_config.pop(key, None)  # 있으면 제거, 없으면 무시

    print(f"📋 사용할 config: {model_config}")

🔄 MaskGIT 로딩 중...
📋 사용할 config: {'vocab_size': 16384, 'seq_len': 256, 'num_classes': 6, 'hidden_dim': 512, 'num_layers': 12, 'num_heads': 8}


In [ ]:
# =============================================================================
# MaskGIT 로딩
# =============================================================================

print("🔄 MaskGIT 로딩 중...")

# 체크포인트 로드
ckpt = torch.load(MASKGIT_CKPT_PATH, map_location=device, weights_only=False)

# Halton MaskGIT 모델 생성 (config가 저장되어 있으면 사용)
maskgit = MaskGITTransformer(**model_config).to(device)
maskgit.load_state_dict(ckpt['model_state_dict'])
maskgit.eval()

🔄 MaskGIT 로딩 중...


MaskGITTransformer(
  (tok_emb): Embedding(16385, 512)
  (pos_emb): Embedding(256, 512)
  (cls_emb): Embedding(7, 512)
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (adaLN_modulation): Sequential(
        (0): SiLU()
        (1): Linear(in_features=512, out_features=3072, bias=True)
      )
      (norm1): RMSNorm()
      (attn): Attention(
        (wq): Linear(in_features=512, out_features=512, bias=False)
        (wk): Linear(in_features=512, out_features=512, bias=False)
        (wv): Linear(in_features=512, out_features=512, bias=False)
        (wo): Linear(in_features=512, out_features=512, bias=False)
        (qk_norm): QKNorm(
          (q_norm): RMSNorm()
          (k_norm): RMSNorm()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): RMSNorm()
      (ffn): SwiGLU(
        (w1): Linear(in_features=512, out_features=1536, bias=False)
        (w2): Linear(in_features=1536, out_features=512, bias=False)
        (w3): Linear(in_featu

## 6. 핵심 함수 정의

In [ ]:
@torch.no_grad()
def encode_to_tokens(images, vqgan_model):
    """LlamaGen 방식 인코딩"""
    _, _, (_, _, indices) = vqgan_model.encode(images)
    batch_size = images.shape[0]
    tokens = indices.view(batch_size, LATENT_SIZE, LATENT_SIZE)
    return tokens

@torch.no_grad()
def decode_from_tokens(tokens, vqgan_model):
    """LlamaGen 방식 디코딩"""
    batch_size = tokens.shape[0]
    tokens_flat = tokens.view(-1)
    tokens_flat = torch.clamp(tokens_flat, 0, CODEBOOK_SIZE - 1)
    embed_dim = vqgan_model.quantize.embedding.weight.shape[1]  # 8

    images = vqgan_model.decode_code(
        tokens_flat,
        shape=(batch_size, embed_dim, LATENT_SIZE, LATENT_SIZE)
    )
    return images

In [ ]:
# =============================================================================
# Cosine Schedule 및 Inpainting 함수
# =============================================================================

def cosine_schedule(t):
    """Cosine masking schedule"""
    return torch.cos(t * torch.pi / 2)

@torch.no_grad()
def inpaint_image(
    original_image,
    mask_region,
    target_class,
    maskgit_model,
    vqgan_model,
    num_steps=8,
    temperature=1.0,
    device='cuda'
):
    """
    원본 이미지의 일부 영역을 특정 결함으로 채움 (Inpainting)

    Args:
        original_image: [1, 3, 256, 256] 원본 이미지 (값 범위 [-1, 1])
        mask_region: 마스킹할 영역의 인덱스 리스트 (0~255)
        target_class: 어떤 결함으로 채울지 (0~5)
        num_steps: 디코딩 스텝 수
        temperature: 샘플링 온도

    Returns:
        result_image: Inpainting 결과 이미지
    """
    maskgit_model.eval()

    # 입력 형태 맞추기
    if original_image.dim() == 3:
        original_image = original_image.unsqueeze(0)
    original_image = original_image.to(device)

    mask_token_id = maskgit_model.mask_token_id
    seq_len = maskgit_model.seq_len

    # mask_region을 텐서로 변환
    if isinstance(mask_region, list):
        mask_region = torch.tensor(mask_region, device=device)

    num_mask_tokens = len(mask_region)

    # 1. 원본 이미지 → 토큰
    original_tokens = encode_to_tokens(original_image, vqgan_model)

    # 2. 마스킹할 영역만 [MASK]로 교체
    tokens = original_tokens.clone().view(1, -1)
    tokens[:, mask_region] = mask_token_id

    # 3. 클래스 라벨
    labels = torch.tensor([target_class], device=device)

    drop_label = torch.zeros(1, dtype=torch.bool, device=device)

    # 4. Iterative decoding (cosine schedule)
    for step in range(num_steps):
        is_masked = (tokens == mask_token_id)
        num_masked = is_masked.sum().item()

        if num_masked == 0:
            break

        # Cosine schedule로 이번 스텝에서 확정할 개수 계산
        t = torch.tensor(step / num_steps, device=device)
        ratio = cosine_schedule(t)
        num_to_keep_masked = (ratio * num_mask_tokens).long().item()
        num_to_unmask = max(1, num_masked - num_to_keep_masked)

        # 예측
        logits = maskgit_model(tokens, labels, drop_label)
        logits = logits / temperature
        probs = F.softmax(logits, dim=-1)
        max_probs, predicted_tokens = probs.max(dim=-1)

        # 마스킹된 위치만 고려
        max_probs = max_probs * is_masked.float()

        # 상위 k개 선택해서 확정
        _, top_k = max_probs.view(-1).topk(num_to_unmask)
        tokens.view(-1)[top_k] = predicted_tokens.view(-1)[top_k]

    # 남은 [MASK] 강제로 채우기
    is_masked = (tokens == mask_token_id)
    if is_masked.any():
        logits = maskgit_model(tokens, labels, drop_label)
        probs = F.softmax(logits / temperature, dim=-1)
        _, predicted_tokens = probs.max(dim=-1)
        tokens[is_masked] = predicted_tokens[is_masked]

    # 5. 토큰 → 이미지
    tokens = tokens.view(1, LATENT_SIZE, LATENT_SIZE)
    result_image = decode_from_tokens(tokens, vqgan_model)

    return result_image

In [ ]:
# ============================================
# Halton Sequence 생성
# ============================================

def halton_sequence(base, n_samples):
    """
    Halton sequence 생성 (1D)

    Args:
        base: 기저 (소수 사용: 2, 3, 5, ...)
        n_samples: 생성할 샘플 수

    Returns:
        list of floats in [0, 1)
    """
    result = []
    for i in range(1, n_samples + 1):
        f = 1.0
        r = 0.0
        n = i
        while n > 0:
            f /= base
            r += f * (n % base)
            n //= base
        result.append(r)
    return result


def build_halton_mask(input_size, n_points=10000):
    """
    2D Halton mask 생성

    Args:
        input_size: 16 (16x16 = 256 토큰)
        n_points: 충분히 큰 수 (중복 제거 후 256개 남음)

    Returns:
        mask: [256, 2] 텐서, 각 행은 (row, col) 좌표
    """
    # 2D Halton sequence (base 2, 3)
    x = halton_sequence(2, n_points)
    y = halton_sequence(3, n_points)

    # [0, 1) → [0, input_size) 스케일링
    coords = []
    seen = set()

    for xi, yi in zip(x, y):
        row = int(xi * input_size)
        col = int(yi * input_size)

        # 범위 체크
        row = min(row, input_size - 1)
        col = min(col, input_size - 1)

        # 중복 제거
        if (row, col) not in seen:
            seen.add((row, col))
            coords.append([row, col])

        # 256개 모이면 종료
        if len(coords) >= input_size ** 2:
            break

    return torch.tensor(coords, dtype=torch.long)


# Halton mask 생성 (한 번만)
HALTON_MASK = build_halton_mask(LATENT_SIZE)
print(f"✅ Halton mask 생성: {HALTON_MASK.shape}")  # [256, 2]
print(f"   처음 5개 좌표: {HALTON_MASK[:5].tolist()}")

✅ Halton mask 생성: torch.Size([256, 2])
   처음 5개 좌표: [[8, 5], [4, 10], [12, 1], [2, 7], [10, 12]]


In [ ]:
# ============================================
# Halton Sampler 클래스
# ============================================

class HaltonSampler:
    """
    Halton-MaskGIT 스타일 샘플러

    특징:
    - Halton sequence로 토큰 순서 결정 (공간적 균일성)
    - CFG (Classifier-Free Guidance) 지원
    - Temperature scheduling
    """

    def __init__(
        self,
        num_steps=32,          # 생성 스텝 수
        cfg_weight=2.0,        # CFG 가중치
        temperature=1.0,       # Softmax 온도
        randomize=True,        # Halton 시퀀스 랜덤화
    ):
        self.num_steps = num_steps
        self.cfg_weight = cfg_weight
        self.temperature = temperature
        self.randomize = randomize
        self.halton_mask = HALTON_MASK

    @torch.no_grad()
    def sample(self, model, vqgan, num_samples, class_labels, device):
        """
        이미지 생성

        Args:
            model: MaskGITTransformer
            vqgan: VQGAN 디코더
            num_samples: 생성할 이미지 수
            class_labels: [num_samples] 클래스 라벨
            device: cuda/cpu

        Returns:
            images: [num_samples, 3, 256, 256] 생성된 이미지
            codes: [num_samples, 16, 16] 생성된 토큰
        """
        model.eval()
        B = num_samples
        H = W = LATENT_SIZE
        seq_len = H * W

        # 1. 전체 [MASK]로 초기화
        code = torch.full((B, H, W), MASK_TOKEN_ID, device=device)

        # 2. 클래스 라벨 준비
        labels = class_labels.to(device)

        # 3. CFG용 라벨 (unconditional)
        drop_label_cond = torch.zeros(B, dtype=torch.bool, device=device)  # 조건부
        drop_label_uncond = torch.ones(B, dtype=torch.bool, device=device)  # 무조건부

        # 4. Halton mask 준비 (샘플마다 랜덤 오프셋)
        if self.randomize:
            offsets = torch.randint(0, seq_len, (B,))
            halton_masks = torch.stack([
                torch.roll(self.halton_mask, shifts=offset.item(), dims=0)
                for offset in offsets
            ])  # [B, 256, 2]
        else:
            halton_masks = self.halton_mask.unsqueeze(0).expand(B, -1, -1)

        halton_masks = halton_masks.to(device)

        # 5. 스텝별 생성
        prev_idx = 0

        for step in tqdm(range(self.num_steps), desc="Sampling", leave=False):
            # 이번 스텝에서 예측할 토큰 수 (arccos schedule)
            ratio = (step + 1) / self.num_steps
            r = 1 - (math.acos(ratio) / (math.pi * 0.5))
            curr_idx = max(step + 1, int(r * seq_len))
            curr_idx = min(curr_idx, seq_len)

            # 이번 스텝에서 예측할 위치들
            positions = halton_masks[:, prev_idx:curr_idx]  # [B, num_new, 2]

            if positions.shape[1] == 0:
                continue

            # Forward (CFG)
            if self.cfg_weight > 0:
                # 조건부 + 무조건부 동시 계산
                code_double = torch.cat([code, code], dim=0)
                labels_double = torch.cat([labels, labels], dim=0)
                drop_double = torch.cat([drop_label_cond, drop_label_uncond], dim=0)

                logits_double = model(code_double, labels_double, drop_double)
                logits_cond, logits_uncond = logits_double.chunk(2, dim=0)

                # CFG 공식
                logits = (1 + self.cfg_weight) * logits_cond - self.cfg_weight * logits_uncond
            else:
                logits = model(code, labels, drop_label_cond)

            # Softmax + 샘플링
            logits = logits.view(B, H, W, -1)  # [B, H, W, vocab+1]
            probs = F.softmax(logits / self.temperature, dim=-1)

            # 각 위치에서 토큰 샘플링
            for b in range(B):
                for pos in positions[b]:
                    row, col = pos[0].item(), pos[1].item()
                    prob = probs[b, row, col, :CODEBOOK_SIZE]  # [MASK] 제외
                    prob = prob / prob.sum()  # 재정규화

                    # Categorical 샘플링
                    sampled_token = torch.multinomial(prob, 1).item()
                    code[b, row, col] = sampled_token

            prev_idx = curr_idx

        # 6. VQGAN 디코딩
        code = torch.clamp(code, 0, CODEBOOK_SIZE - 1)
        images = decode_from_tokens(code, vqgan)
        images = torch.clamp(images, -1, 1)

        model.train()
        return images, code


# Sampler 생성
sampler = HaltonSampler(
    num_steps=32,
    cfg_weight=2.0,
    temperature=1.0,
    randomize=True
)

print("✅ Halton Sampler 생성 완료")
print(f"   Steps: {sampler.num_steps}")
print(f"   CFG Weight: {sampler.cfg_weight}")
print(f"   Temperature: {sampler.temperature}")

✅ Halton Sampler 생성 완료
   Steps: 32
   CFG Weight: 2.0
   Temperature: 1.0


In [ ]:
# =============================================================================
# 이미지 전처리/후처리 함수
# =============================================================================

def preprocess_image(pil_image):
    """PIL Image → 모델 입력 텐서"""
    transform = T.Compose([
        T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        T.Grayscale(num_output_channels=3),
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    if pil_image.mode != 'RGB':
        pil_image = pil_image.convert('RGB')

    tensor = transform(pil_image).unsqueeze(0)  # [1, 3, 256, 256]
    return tensor.to(device)


def tensor_to_pil(tensor):
    """텐서 → PIL Image"""
    tensor = tensor.cpu().detach()

    if tensor.dim() == 4:
        tensor = tensor[0]

    # [-1, 1] → [0, 1]
    tensor = (tensor + 1) / 2
    tensor = tensor.clamp(0, 1)

    # [C, H, W] → [H, W, C]
    numpy_img = tensor.permute(1, 2, 0).numpy()
    numpy_img = (numpy_img * 255).astype(np.uint8)

    return Image.fromarray(numpy_img)


def visualize_mask_on_image(pil_image, mask_indices, color=(255, 0, 0), alpha=0.4):
    """이미지 위에 마스크 영역 시각화"""
    img_np = np.array(pil_image.resize((IMAGE_SIZE, IMAGE_SIZE))).astype(np.float32)

    # 마스크 생성
    mask = np.zeros((LATENT_SIZE, LATENT_SIZE), dtype=np.float32)
    for idx in mask_indices:
        y = idx // LATENT_SIZE
        x = idx % LATENT_SIZE
        mask[y, x] = 1.0

    # 이미지 크기로 업스케일
    scale = IMAGE_SIZE // LATENT_SIZE
    mask_upscaled = np.kron(mask, np.ones((scale, scale)))
    mask_3d = np.stack([mask_upscaled] * 3, axis=-1)

    # 오버레이
    color_overlay = np.array(color, dtype=np.float32).reshape(1, 1, 3)
    overlay = img_np * (1 - mask_3d * alpha) + color_overlay * mask_3d * alpha
    overlay = np.clip(overlay, 0, 255).astype(np.uint8)

    return Image.fromarray(overlay)

## 7. Gradio 앱 정의

In [ ]:
# =============================================================================
# Gradio 앱
# =============================================================================

import gradio as gr

# Inpainting 함수 (Gradio용)
def gradio_inpaint(image, defect_type, mask_preset, num_steps, temperature):
    """
    Gradio에서 호출하는 Inpainting 함수
    """
    if image is None:
        return None, None, None, "❌ 이미지를 업로드해주세요!"

    try:
        # PIL Image로 변환
        if isinstance(image, np.ndarray):
            pil_image = Image.fromarray(image)
        else:
            pil_image = image

        # 클래스 인덱스 추출
        class_idx = CLASS_NAMES_KR.index(defect_type)

        # 마스크 인덱스
        mask_indices = get_mask_preset(mask_preset)

        # 전처리
        input_tensor = preprocess_image(pil_image)

        # VQGAN 재구성 (품질 확인용)
        with torch.no_grad():
            tokens = encode_to_tokens(input_tensor, vqgan)
            reconstructed = decode_from_tokens(tokens, vqgan)

        # Inpainting
        inpainted = inpaint_image(
            original_image=input_tensor,
            mask_region=mask_indices,
            target_class=class_idx,
            maskgit_model=maskgit,
            vqgan_model=vqgan,
            num_steps=int(num_steps),
            temperature=temperature,
            device=device,
        )

        # 결과 변환
        original_pil = tensor_to_pil(input_tensor)
        reconstructed_pil = tensor_to_pil(reconstructed)
        inpainted_pil = tensor_to_pil(inpainted)

        # 마스크 시각화
        mask_preview = visualize_mask_on_image(original_pil, mask_indices)

        status = f"✅ 완료! 결함: {CLASS_NAMES[class_idx]}, 마스크: {len(mask_indices)}개 토큰"

        return mask_preview, reconstructed_pil, inpainted_pil, status

    except Exception as e:
        return None, None, None, f"❌ 에러: {str(e)}"

In [ ]:
# =============================================================================
# Gradio 인터페이스 생성 및 실행
# =============================================================================

# 마스크 프리셋 목록
MASK_PRESETS = [
    "중앙 (작음) 6x6",
    "중앙 (큼) 8x8",
    "좌상단 6x6",
    "우하단 6x6"
]

# CSS 스타일
css = """
.gradio-container { font-family: 'Noto Sans KR', sans-serif; }
h1 { text-align: center; }
"""

# Gradio Blocks 인터페이스
with gr.Blocks(css=css, title="금속 결함 합성기") as demo:

    gr.Markdown("""
# 🔧 금속 결함 합성기 (Metal Defect Synthesizer)

**NEU-DET / X-SDD / SD-saliency-900** 데이터셋 기반 금속 표면 결함 Inpainting 도구

기존 이미지의 특정 영역에 원하는 결함을 추가합니다.
""")
    with gr.Row():
        with gr.Column(scale=1):
            # 입력
            input_image = gr.Image(
                label="원본 이미지 업로드",
                type="pil",
            )

            defect_dropdown = gr.Dropdown(
                choices=CLASS_NAMES_KR,
                value=CLASS_NAMES_KR[5],  # scratches
                label="추가할 결함 타입",
            )

            mask_dropdown = gr.Dropdown(
                choices=MASK_PRESETS,
                value=MASK_PRESETS[0],
                label="마스크 영역",
            )

            with gr.Row():
                steps_slider = gr.Slider(
                    minimum=4, maximum=16, value=8, step=1,
                    label="디코딩 스텝",
                )
                temp_slider = gr.Slider(
                    minimum=0.5, maximum=2.0, value=1.0, step=0.1,
                    label="Temperature",
                )

            inpaint_btn = gr.Button("🚀 Inpainting 실행", variant="primary")

        with gr.Column(scale=2):
            # 출력
            status_text = gr.Textbox(label="상태", interactive=False)

            with gr.Row():
                mask_preview = gr.Image(label="마스크 미리보기")
                reconstructed_output = gr.Image(label="VQGAN 재구성")
                inpainted_output = gr.Image(label="Inpainting 결과")

    # 버튼 이벤트
    inpaint_btn.click(
        fn=gradio_inpaint,
        inputs=[input_image, defect_dropdown, mask_dropdown, steps_slider, temp_slider],
        outputs=[mask_preview, reconstructed_output, inpainted_output, status_text],
    )

    # 하단 정보
    gr.Markdown("""
---
### 📊 지원 결함 타입

| 결함 | 시각적 특징 |
|------|------------|
| 개재물 | 표면에 박힌 어두운 점이나 덩어리. 금속 내부 불순물이 드러난 것 |
| 피트홀 | 표면에 움푹 파인 작은 구멍들. 펀칭 실수나 압연 중 생긴 함몰 |
| 스크래치/주름 | 길게 긁힌 선 자국이나 접혀서 생긴 주름 |
| 얼룩 | 물, 기름 등이 마르면서 생긴 불규칙한 밝거나 어두운 영역 |
| 균열 | 거미줄처럼 퍼진 미세 균열이나 용접/절단 부위의 틈 |
| 접힘/스케일 | 압연 중 산화막이 눌려 들어간 어두운 줄무늬나 접힌 자국 |

### 💡 Tips
- **Temperature**: 낮을수록 안정적, 높을수록 다양함
- **스텝 수**: 많을수록 품질↑, 속도↓
""")

/tmp/ipython-input-2325637056.py:20: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, title="금속 결함 합성기") as demo:


## 8. 🚀 앱 실행!

In [ ]:
# =============================================================================
# 🚀 Gradio 앱 실행
# =============================================================================

# share=True: 공개 URL 생성 (72시간 유효)
# debug=True: 에러 상세 출력

demo.launch(
    share=True,
    debug=True,
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cbe2331d7e6f2a819b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cbe2331d7e6f2a819b.gradio.live
